<a href="https://colab.research.google.com/github/aneeshoberoi/churnhw/blob/main/ChurnNoteBook_Final_FullDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
from collections import OrderedDict

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option("display.precision",2)

In [6]:
df=pd.read_csv('https://raw.githubusercontent.com/aneeshoberoi/churnhw/main/churn_homework%20-%20homework_orders.csv?token=AP7KM2RBFAH333CPSXM4ILC7SV6WI', sep=",")

In [15]:
raw=df.head(100)
raw.set_index('order_id')

,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code
order_id,,,,,,,,,,
47433960,iosapp,2019-03-13T08:02:07.000Z,Delivery,credit,0,16,13315067198,24597,MD,21162
47434359,iosapp,2019-03-13T08:11:38.000Z,Delivery,credit,0,25,15557129447,64392,NY,12944
47437194,partner_website,2019-03-13T09:07:31.000Z,Delivery,cash,0,43,25689365012,33699,NJ,8007
47442942,partner_website,2019-03-13T11:01:04.000Z,Delivery,credit,0,28,25421112251,19002,WI,53235
47443107,partner_website,2019-03-13T11:04:56.000Z,Delivery,credit,0,23,18529185191,34911,MA,2122
...,...,...,...,...,...,...,...,...,...,...
47571438,iosapp,2019-03-14T20:33:11.000Z,Delivery,credit,0,19,18312751742,57072,PA,19382
47574693,partner_website,2019-03-15T07:32:12.000Z,Delivery,cash,0,17,6103411793,15639,CT,6484
47575101,androidapp,2019-03-15T07:51:01.000Z,Delivery,credit,0,37,27267518882,30054,NJ,7921


In [ ]:
# assigning to working copy for indicator variables

# **Basic Statistics**

In [12]:
raw.customer.nunique()

87

In [13]:
raw.restaurant_total.sum()

2909.73

In [16]:
raw.order_id.nunique()

100

# ***Data Cleaning***

In [17]:
# assigning to working copy for indicator variables
test=raw

In [18]:
# Remove zero dollar txns
test=test[test['restaurant_total']>0]

In [ ]:
# Variable Conversion
test['date_purchased']=pd.to_datetime(test['date_purchased'])
test['dateonly_purchased']=test['date_purchased'].dt.date
test['houronly_purchased']=test['date_purchased'].dt.hour
test['datehour_purchased']=test['dateonly_purchased'].astype(str) + "-" + test['houronly_purchased'].astype(str)

test['restaurant_total']=test['restaurant_total'].astype(float)

In [20]:
# Removing Duplicate Txns - i.e. those made within the same hour at the same shop by a given customer. Keeping last txn only.
test=test.drop_duplicates(subset=['customer', 'shops_id', 'datehour_purchased'], keep='last')


In [21]:
# Checking Impact of cleanup

test.restaurant_total.sum()

2693.3500000000004

In [22]:
test.order_id.count()

90

# ***Data Prep - Indicators***

In [23]:
 #Enumerating Purchases
 test.sort_values(by=['customer','date_purchased'])
 
test = test.assign(order_enum=test.groupby('customer')['datehour_purchased'].cumcount()+1)
test['total_order']=test.groupby('customer')['order_id'].transform('count')

In [24]:
test['bin_orders']=np.where(test['total_order']<7,test['order_enum'],7)

In [25]:
# Calculating Journey Data

test['time_between_purch']=test.groupby(['customer'])['date_purchased'].diff()
test['spend_growth']=test.groupby(['customer'])['restaurant_total'].diff()


# Preparing Zip-Cust Level Variables




In [262]:
#Assigning raw dataset to working copy for this section
shopszip=test[['postal_code', 'shops_id','customer','total_order']].set_index('postal_code')
shopszip=shopszip.drop_duplicates(subset=['customer', 'shops_id'], keep='last')

In [263]:
# calculating shops and customers per postal code
shopszip['shops_per_zip']=shopszip.groupby('postal_code').shops_id.nunique()
shopszip['cust_per_zip']=shopszip.groupby('postal_code').customer.nunique()
shopszip['shops_per_cust_per_zip']=shopszip['shops_per_zip']/shopszip['cust_per_zip']

In [264]:
# Of customers placing orders in a given zip, what # and  % have placed more than 2 orders in total?
temp=shopszip[shopszip['total_order']>2]
shopszip['zip_cust_2_more']=temp.groupby('postal_code')['customer'].count()
shopszip['zip_pct_cust_2_more']=shopszip['zip_cust_2_more']*100/shopszip['cust_per_zip']

In [242]:
shopszip[shopszip['shops_id']==27246]

,shops_id,customer,total_order,shops_per_zip,cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,shops_per_cust_per_zip
postal_code,,,,,,,,
98101,27246,6196987925,3,1,1,1,100,1


# ***Preparing Shop-Cust Level Variables***

In [133]:
#Total customers served by each shop

In [285]:
shopscust=shopszip.set_index('shops_id')
shopscust=shopscust.drop(columns=['shops_per_zip','cust_per_zip','zip_cust_2_more', 'zip_pct_cust_2_more','shops_per_cust_per_zip'] )

# How many customers have placed at least one order at a given shop?

shopscust['cust_per_shop']=shopscust.groupby('shops_id').customer.nunique()

In [286]:
# Of customers placing orders at a given shop, what # and % have placed > 2 total orders/
t=shopscust[shopscust['total_order']>2]
shopscust['shop_cust_2_more']=t.groupby('shops_id')['customer'].count()
shopscust['shop_pct_cust_2_more']=shopscust['shop_cust_2_more']*100/shopscust['cust_per_shop']

# ***Preparing Shop-Sales Level Variables***

In [344]:
shopsales=test[['shops_id','customer','order_id','restaurant_total']] 

In [ ]:
shopsales['total_shop_sales']=shopsales.groupby('shops_id').restaurant_total.transform('sum')
shopsales['total_shop_orders']=shopsales.groupby('shops_id').order_id.transform('count')

In [354]:
shopsales=shopsales.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','order_id','restaurant_total'])

In [360]:
shopsales['sales_qunitiles']=pd.qcut(shopsales['total_shop_sales'],5,labels=False)
shopsales['order_qunitiles']=pd.qcut(shopsales['total_shop_orders'].rank(method='first'),5,labels=False)

#pd.qcut(df['a'].rank(method='first'), 3, labels=False)

In [361]:
shopsales

,shops_id,total_shop_sales,total_shop_orders,sales_qunitiles,order_qunitiles
0,24597,16,1,0,0
1,64392,25,1,2,0
2,33699,43,1,4,0
3,19002,28,1,2,0
4,34911,23,1,1,0
...,...,...,...,...,...
95,57072,19,1,1,4
96,15639,17,1,0,4
97,30054,37,1,3,4
98,40359,23,1,2,4


# ***Combining to create final analytical dataset***

In [288]:
shopscust_final=shopscust.reset_index()
shopscust_final=shopscust_final.drop_duplicates(subset=['shops_id'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')


In [323]:
shopscust_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 24597 to 51831
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cust_per_shop         85 non-null     int64  
 1   shop_cust_2_more      1 non-null      float64
 2   shop_pct_cust_2_more  1 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 2.7 KB


In [324]:
# Merging shopscust with original dataset

test2=test.merge(shopscust_final,left_on='shops_id',right_on='shops_id') 


In [327]:
#test2[test2['customer']==6196987925]

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,bin_orders,time_between_purch,spend_growth,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more
22,47474511,slicelife,2019-03-13 16:33:51+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-13,16,2019-03-13-16,1,3,1,NaT,nan,1,1,100
23,47513940,slicelife,2019-03-14 13:15:11+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-14,13,2019-03-14-13,2,3,2,0 days 20:41:20,0,1,1,100
24,47566653,slicelife,2019-03-14 18:44:08+00:00,Delivery,cash,0,19,6196987925,27246,WA,98101,2019-03-14,18,2019-03-14-18,3,3,3,0 days 05:28:57,-2,1,1,100


In [ ]:
# Merging shopszip with original dataset
shopszip_final=shopszip.reset_index()
shopszip_final=shopszip_final.drop_duplicates(subset=['postal_code'], keep='last').drop(columns=['customer','total_order']).set_index('shops_id')
shopszip_final.info()
#shopscustzip_final=shopscustzip.drop(columns=['customer','total_order'])

In [335]:
shopszip_final.set_index('postal_code')
test3=test2.merge(shopszip_final,left_on='postal_code',right_on='postal_code') 

In [362]:
# Merging shopsales with original dataset
test4=test3.merge(shopsales,left_on='shops_id',right_on='shops_id')

In [364]:
test4[test4['customer']==6196987925]

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,bin_orders,time_between_purch,spend_growth,cust_per_shop,shop_cust_2_more,shop_pct_cust_2_more,shops_per_zip,cust_per_zip,shops_per_cust_per_zip,zip_cust_2_more,zip_pct_cust_2_more,total_shop_sales,total_shop_orders,sales_qunitiles,order_qunitiles
22,47474511,slicelife,2019-03-13 16:33:51+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-13,16,2019-03-13-16,1,3,1,NaT,nan,1,1,100,1,1,1,1,100,61,3,4,4
23,47513940,slicelife,2019-03-14 13:15:11+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-14,13,2019-03-14-13,2,3,2,0 days 20:41:20,0,1,1,100,1,1,1,1,100,61,3,4,4
24,47566653,slicelife,2019-03-14 18:44:08+00:00,Delivery,cash,0,19,6196987925,27246,WA,98101,2019-03-14,18,2019-03-14-18,3,3,3,0 days 05:28:57,-2,1,1,100,1,1,1,1,100,61,3,4,4
